# MLP (Neural Network in sklearn)

Alternative zu tensorflow kann man auch MLP von sklearn brauchen.
Wie zeigen wir in diesem Notebook.

In [1]:
import pickle
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

import matplotlib.pyplot as plt

In [2]:
def plot_confusion_matrix(y_true, y_pred):
    # Takes true and predicted labels (arrays) as input. Based on this, calculates and plots a confusion matrix.
    labels = np.unique(y_true)
    fig = plt.figure(figsize=(len(labels), len(labels)))
    ConfusionMatrixDisplay.from_predictions(
        y_true, y_pred, 
        normalize='all', 
        xticks_rotation='vertical', 
        ax=fig.gca(), 
        cmap="BuPu"
    )
    plt.show()

# Prepare data

In [3]:
# Load the data and split into features and labels
with open('../data/train.pkl', 'rb') as f:
    data_train = pickle.load(f)
X_data = data_train["images"]
y_data = data_train["labels"]

In [4]:
# Split features and labels into train (X_train, y_train) and validation set (X_val, y_val).
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, random_state=42)

In [5]:
%%time

nn = Pipeline([
    ('std', StandardScaler()), 
    ('clf', MLPClassifier(hidden_layer_sizes=(1024,), max_iter=10, verbose=True))
], verbose=True)

nn.fit(X_train.reshape(-1, 32 * 32 * 3), y_train)

y_train_hat = nn.predict(X_train.reshape(-1, 32 * 32 * 3))
print("Train: ", accuracy_score(y_train_hat, y_train))

y_val_hat = nn.predict(X_val.reshape(-1, 32 * 32 * 3))
print("Val:   ", accuracy_score(y_val_hat, y_val))

[Pipeline] ............... (step 1 of 2) Processing std, total=   2.3s
Iteration 1, loss = 2.46135364
Iteration 2, loss = 1.63304685
Iteration 3, loss = 1.40471575
Iteration 4, loss = 1.26712156
Iteration 5, loss = 1.19415934
Iteration 6, loss = 1.13212214
Iteration 7, loss = 1.09185845
Iteration 8, loss = 1.00326744
Iteration 9, loss = 0.98511813
Iteration 10, loss = 0.96037340
[Pipeline] ............... (step 2 of 2) Processing clf, total= 3.1min


/Users/ivo/.virtualenvs/mllab/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


Train:  0.7219733333333334
Val:    0.49624
CPU times: user 20min 6s, sys: 4min 38s, total: 24min 45s
Wall time: 3min 13s


# Predict classes for test set

If we are happy with the performance of our model on the validation set, we can apply it to the test set.

In [9]:
# Daten laden und aufteilen in Features und Labels
with open('../data/test.pkl', 'rb') as f:
    data_test = pickle.load(f)
    
X_test = data_test["images"]
y_test = data_test["labels"]

In [10]:
y_test_pred = nn.predict(X_test.reshape(-1, 32 * 32 * 3))

Evaluieren wir unsere test-set predictions

In [12]:
accuracy_score(y_test, y_test_pred)

0.4861